<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BigData---Final-Project" data-toc-modified-id="BigData---Final-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BigData - Final Project</a></span><ul class="toc-item"><li><span><a href="#Loading-The-Data" data-toc-modified-id="Loading-The-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Loading The Data</a></span></li><li><span><a href="#Exploring-The-Data" data-toc-modified-id="Exploring-The-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Exploring The Data</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Analysis</a></span></li></ul></li></ul></div>

# BigData - Final Project

__AUTHORS__:
  - Théo Perinet (22172 - theo.perinet)
  - Mathieu Rivier (23553 - mathieu.rivier)
  - Marc Monteil (23742 - marc.monteil)

###### To Use when you are on google collab
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz  
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

###### TO USE WHEN YOU ARE ON GOOGLE COLLAB
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"
import findspark
findspark.init()

from google.colab import drive
drive.mount('/content/drive')

## Loading The Data

In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
spark_application_name = "WannaFlop_Project"

In [3]:
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 09:35:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Exploring The Data

In [4]:
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, StructType,StructField
from pyspark.sql.functions import desc
import pyspark.sql.functions as func

In [59]:
class read_info(object):
    def __init__(self, file_path, header=False, delimiter=';', schema=None):
        self.file_path = file_path
        self.header = header
        self.delimiter = delimiter
        self.schema = schema

        self.df = self._load_df()

        #self.df_abstract = self._get_df_abstract()

    def __repr__(self):
        return f"{self._nb_rows()} \n{self.df.printSchema()} \n{self.get_df_abstract()}\n {self.show_missing()}\n{self._get_stats()}"

    def show_missing(self):
        print("Missing Data per column:")
        self._count_missing().show()

    def _get_num_cols(self):
        num_cols = [
            f.name for f in self.df.schema.fields
            if isinstance(f.dataType, DoubleType) or
            isinstance(f.dataType, IntegerType)
        ]
        
        return num_cols
    def _get_rounded_df(self):
        rounded_df = self.df
        dbl_cols = self._get_num_cols()
        for col in dbl_cols:
            rounded_df = rounded_df.withColumn(col, func.round('high'))

        return rounded_df

    def get_df_abstract(self):
        rounded_df = self._get_rounded_df()

        # First 40 rows
        print("First 40 rows:")
        rounded_df.show(40)

        # Last 40 rows
        print("Last 40 rows:")
        rounded_df = rounded_df.withColumn(
            "index", monotonically_increasing_id()
        )
        rounded_df.orderBy(desc("index")).drop("index").show(40)

    def _get_periodicity(self):
        self.df['data'][0]

    def _nb_rows(self):
        # Number of total rows
        print("Number of rows: " + str(self.df.count()) + "\n")

    def _handle_csv(self):
        '''
        @description: Read the csv file and return a Spark DataFrame

        @arg csv_file_path: Path to the csv file
        @arg header: boolean whether to load a header or not
        @arg delimiter: which delimiter to use by default
        '''
        return spark.read.option("inferSchema", "true").option("nullValue", "null").csv(
            self.file_path,
            sep=self.delimiter,
            schema=self.schema,
            header=self.header,
        )
    
    def _handle_json(self):
        return spark.read.json(self.file_path)

    def _load_df(self):
        ####### ADD TRY CATCH #####
        extension = self.file_path.split(".")[-1]

        df = None
        if extension == 'json':
            df = self._handle_json()
        elif extension == 'csv':
            df = self._handle_csv()

        return df

    def _count_missing(self):
        cols = self.df.columns
        cols.remove('Date')
        return self.df.select(
            [
                count(when(isnan(c) | col(c).isNull(), c)).alias(c)
                for c in cols
            ]
        )
        #.show()
        
    def _get_stats(self):
        self.df.summary().show()

In [60]:
print(h)
h.remove('Date')
print(h)

NameError: name 'h' is not defined

In [5]:
amzn_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', IntegerType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])

In [65]:
AMZN = read_info('stocks_data/AMAZON.csv', header=True, delimiter=',', schema=amzn_schema)

NameError: name 'spark' is not defined

In [66]:
print(AMZN)

In [67]:
##### A FAIRE !!!! UN SCHEMA !!!!!

In [68]:
AMZN.get_df_abstract()

AttributeError: 'Stock' object has no attribute 'get_df_abstract'

In [69]:
AMZN.show_missing()

AttributeError: 'Stock' object has no attribute 'show_missing'

In [70]:
AMZN._get_stats()

AttributeError: 'Stock' object has no attribute '_get_stats'

In [71]:
AMZN.df.withColumn("test", 
              func.datediff(AMZN.df["date"][0], AMZN.df["date"][1])).show()

AnalysisException: Can't extract value from date#4536: need struct type but got date

In [72]:
AMZN.df["Date"].getItem(2)

Column<'Date[2]'>

In [73]:
AMZN.df.first()['Date']

datetime.date(2017, 1, 3)

In [74]:
AMZN.df.__get_item(0)

AttributeError: 'DataFrame' object has no attribute '__get_item'

In [75]:
AMZN.df.second()['Date']

AttributeError: 'DataFrame' object has no attribute 'second'

In [76]:
func.getrows(AMZN.df, rownums=[0, 2]).collect()

AttributeError: module 'pyspark.sql.functions' has no attribute 'getrows'

In [ ]:
AMZN.df[0].__getitem__("Date").first()

In [ ]:
AMZN.df[0]

In [ ]:
AMZN.df.select('Date').show()

In [ ]:
AMZN.df


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window



my_window = Window.partitionBy().orderBy("Date")

df = AMZN.df.withColumn("prev_value", F.lag(AMZN.df.Date).over(my_window))
df = df.withColumn("diff", F.when(F.isnull(F.datediff(df.Date, df.prev_value)), 0)
                              .otherwise(F.datediff(df.Date, df.prev_value)))

In [ ]:
df.select("diff").show()

In [ ]:
from pyspark.sql.functions import mean

In [ ]:
df.select(mean('diff')).first()[0]

In [ ]:
AMZN.df.stat.corr('High', 'Low')

TODO: Create function to compute per month week year

In [ ]:
AMZN.df.select(mean ("Close")).first()[0]

In [ ]:
def get_col_mean(df, col):
    return df.select(mean (col)).first()[0]

In [ ]:
get_col_mean(AMZN.df, "Close")

In [ ]:
AMZN.df.groupBy(func.weekofyear("day").alias("date_by_week")).agg(sum("Date"))

In [ ]:
AMZN.df.groupBy(func.weekofyear("day").alias("date_by_week")).agg(sum("Close")).orderBy("date_by_week").show()

In [ ]:
AMZN.df.withColumn("Date",func.date_sub(func.next_day(col("Date"),"sunday"),7)).groupBy("Date").agg(sum("Close").cast("int").alias("Close_total")).orderBy("week_strt_day").show()


In [ ]:
AMZN.df.groupBy("Date").select("Close").show()

In [ ]:
AMZN.df.groupBy(func.month("Date").alias("hour")).agg(mean("Close").alias("close_mean")).show()

In [ ]:
AMZN.df.groupBy(func.year("Date").alias("hour")).agg(mean("Close").alias("close_mean")).show()

In [ ]:
def 

In [ ]:
get_avg(AMZN.df, "Close", func.year)

In [ ]:
get_avg(AMZN.df, "Open", func.year)

In [ ]:
class Exploration(object):
    def __init__(self, df):
        self.df = df

    def get_oc_avg(self, fun):
        close = self._compute_avg(self.df, "Close", fun)
        opening = self._compute_avg(self.df, "Open", fun)

        return close.join(
            opening, opening.Open_new_time == close.Close_new_time, "inner"
        ).orderBy("Close_new_time").select(
            close.Close_new_time, close.Close_mean, opening.Open_mean
        )

    def _compute_avg(self, df, col, fun):
        return df.groupBy(fun("Date").alias(col + "_new_time")).agg(
            mean(col).alias(col + "_mean")
        )

In [ ]:
exAMZN = Exploration(AMZN.df)

In [ ]:
exAMZN.get_oc_avg(func.month).show()

In [ ]:
exAMZN.get_oc_avg(func.year).show()

In [ ]:
def get_price_change(period=None):
    df = AMZN.df
    if period:
        df= exAMZN.get_oc_avg(period)
   
    return  df.withColumn('diff', ( df['Close_mean'] - df['Open_mean'] ))

In [ ]:
get_price_change(func.month).show()

In [ ]:
get_price_change(func.year).show()

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from stock_info import Stock

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 15:34:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, StructType,StructField

In [6]:
amzn_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', IntegerType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])

In [7]:
AMZN = Stock('stocks_data/MICROSOFT.csv', header=True, delimiter=',', schema=amzn_schema)

In [8]:
AMZN.explore

root
 |-- Date: date (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)

First 40 rows:
+----------+----+----+----+-----+------+---------+------------+
|      Date|High| Low|Open|Close|Volume|Adj Close|company_name|
+----------+----+----+----+-----+------+---------+------------+
|2017-01-03|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-04|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-05|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-06|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-09|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-10|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-11|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-12|63.0|63.0|63.0| 63.0|  63

Stock Stats:
+-------+-----+-----+-----+-----+------+---------+------------+
|summary| High|  Low| Open|Close|Volume|Adj Close|company_name|
+-------+-----+-----+-----+-----+------+---------+------------+
|  count|987.0|987.0|987.0|987.0| 987.0|    987.0|         987|
|   mean|124.0|124.0|124.0|124.0| 124.0|    124.0|        null|
| stddev| 46.0| 46.0| 46.0| 46.0|  46.0|     46.0|        null|
|    min| 63.0| 63.0| 63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|    25%| 86.0| 86.0| 86.0| 86.0|  86.0|     86.0|        null|
|    50%|111.0|111.0|111.0|111.0| 111.0|    111.0|        null|
|    75%|152.0|152.0|152.0|152.0| 152.0|    152.0|        null|
|    max|233.0|233.0|233.0|233.0| 233.0|    233.0|   MICROSOFT|
+-------+-----+-----+-----+-----+------+---------+------------+

Missing Data per column:
+----+---+----+-----+------+---------+------------+
|High|Low|Open|Close|Volume|Adj Close|company_name|
+----+---+----+-----+------+---------+------------+
|   0|  0|   0|    0|   987|        0


            None

            None

            None

            None

            None

In [9]:
AMZN.explore._nb_rows()

Number of rows: 987



In [10]:
AMZN.explore._print_schema()

root
 |-- Date: date (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)



In [11]:
AMZN.explore.get_df_abstract()

First 40 rows:
+----------+----+----+----+-----+------+---------+------------+
|      Date|High| Low|Open|Close|Volume|Adj Close|company_name|
+----------+----+----+----+-----+------+---------+------------+
|2017-01-03|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-04|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-05|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-06|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-09|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-10|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-11|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-12|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-13|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-17|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-18|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-19|63.0|63.0|63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|2017-01-20|63.0|63.0|63.

In [12]:
AMZN.explore.get_missing()

Missing Data per column:
+----+---+----+-----+------+---------+------------+
|High|Low|Open|Close|Volume|Adj Close|company_name|
+----+---+----+-----+------+---------+------------+
|   0|  0|   0|    0|   987|        0|           0|
+----+---+----+-----+------+---------+------------+



In [29]:
AMZN.explore.get_stats()

Stock Stats:
+-------+-----+-----+-----+-----+------+---------+------------+
|summary| High|  Low| Open|Close|Volume|Adj Close|company_name|
+-------+-----+-----+-----+-----+------+---------+------------+
|  count|987.0|987.0|987.0|987.0| 987.0|    987.0|         987|
|   mean|124.0|124.0|124.0|124.0| 124.0|    124.0|        null|
| stddev| 46.0| 46.0| 46.0| 46.0|  46.0|     46.0|        null|
|    min| 63.0| 63.0| 63.0| 63.0|  63.0|     63.0|   MICROSOFT|
|    25%| 86.0| 86.0| 86.0| 86.0|  86.0|     86.0|        null|
|    50%|111.0|111.0|111.0|111.0| 111.0|    111.0|        null|
|    75%|152.0|152.0|152.0|152.0| 152.0|    152.0|        null|
|    max|233.0|233.0|233.0|233.0| 233.0|    233.0|   MICROSOFT|
+-------+-----+-----+-----+-----+------+---------+------------+



## Analysis

In [30]:
AMZN.analysis.get_oc_avg("month").show()

+-------+------------------+------------------+
|   Date|        Close_mean|         Open_mean|
+-------+------------------+------------------+
|2017-01| 63.19199962615967|63.185500144958496|
|2017-02| 64.11368440326892| 64.13473711515728|
|2017-03| 64.84130494490914| 64.76434906669284|
|2017-04| 66.17157946134868| 66.23894781815379|
|2017-05| 68.91727308793502| 68.82818222045898|
|2017-06|  70.5181815407493| 70.56181820956144|
|2017-07| 72.01050033569337| 71.84349975585937|
|2017-08| 72.81695755668308|  72.7156518023947|
|2017-09| 74.34450073242188|  74.3654998779297|
|2017-10| 77.93954571810636| 77.89318119395863|
|2017-11| 83.71761903308686| 83.64523824055989|
|2017-12|   84.758500289917| 84.83599967956543|
|2018-01| 90.07523781912667| 89.96666681198846|
|2018-02| 91.36789462440892| 91.43736829255757|
|2018-03| 92.89904748825799| 93.23047601609002|
|2018-04| 93.21476164318267| 93.53095318022228|
|2018-05| 96.98136381669478| 96.63272718949752|
|2018-06|100.56190454392205|100.66571517

In [36]:
AMZN.analysis.get_oc_avg("year").show()

+----+------------------+------------------+
|Date|        Close_mean|         Open_mean|
+----+------------------+------------------+
|2017| 71.98402421502954| 71.95430287516925|
|2018|101.03398411967365|101.12235092831799|
|2019|130.38202400813026|130.33904787093874|
|2020| 190.8616180419922|190.76480678836674|
+----+------------------+------------------+



In [40]:
AMZN.analysis.get_price_change("month").show()

+-------+------------------+------------------+--------------------+
|   Date|        Close_mean|         Open_mean|                diff|
+-------+------------------+------------------+--------------------+
|2017-01| 63.19199962615967|63.185500144958496|0.006499481201174717|
|2017-02| 64.11368440326892| 64.13473711515728|  -0.021052711888359|
|2017-03| 64.84130494490914| 64.76434906669284| 0.07695587821629601|
|2017-04| 66.17157946134868| 66.23894781815379|-0.06736835680510467|
|2017-05| 68.91727308793502| 68.82818222045898| 0.08909086747603112|
|2017-06|  70.5181815407493| 70.56181820956144|-0.04363666881214101|
|2017-07| 72.01050033569337| 71.84349975585937| 0.16700057983399574|
|2017-08| 72.81695755668308|  72.7156518023947| 0.10130575428837574|
|2017-09| 74.34450073242188|  74.3654998779297|-0.02099914550781...|
|2017-10| 77.93954571810636| 77.89318119395863| 0.04636452414773373|
|2017-11| 83.71761903308686| 83.64523824055989| 0.07238079252697105|
|2017-12|   84.758500289917| 84.83

In [41]:
AMZN.analysis.get_price_change("year").show()

+----+------------------+------------------+--------------------+
|Date|        Close_mean|         Open_mean|                diff|
+----+------------------+------------------+--------------------+
|2017| 71.98402421502954| 71.95430287516925|  0.0297213398602878|
|2018|101.03398411967365|101.12235092831799|-0.08836680864433788|
|2019|130.38202400813026|130.33904787093874| 0.04297613719151627|
|2020| 190.8616180419922|190.76480678836674| 0.09681125362544662|
+----+------------------+------------------+--------------------+



In [ ]:
(AMZN.df.withColumn('Date', func.date_format(func.col('Date'), 'yyyy-MM'))
.groupBy('Date')
.agg(func.mean('Close'))
).show()

In [42]:
AMZN.analysis._compute_avg(AMZN.df, "Close", "month").show(50)

+-------+------------------+
|   Date|        Close_mean|
+-------+------------------+
|2017-01| 63.19199962615967|
|2017-02| 64.11368440326892|
|2017-03| 64.84130494490914|
|2017-04| 66.17157946134868|
|2017-05| 68.91727308793502|
|2017-06|  70.5181815407493|
|2017-07| 72.01050033569337|
|2017-08| 72.81695755668308|
|2017-09| 74.34450073242188|
|2017-10| 77.93954571810636|
|2017-11| 83.71761903308686|
|2017-12|   84.758500289917|
|2018-01| 90.07523781912667|
|2018-02| 91.36789462440892|
|2018-03| 92.89904748825799|
|2018-04| 93.21476164318267|
|2018-05| 96.98136381669478|
|2018-06|100.56190454392205|
|2018-07| 104.6385730561756|
|2018-08|108.68434740149456|
|2018-09| 112.2378945601614|
|2018-10|  109.207391490107|
|2018-11|107.29809606642951|
|2018-12|104.39368358411286|
|2019-01|104.13523828415643|
|2019-02|107.92789419073807|
|2019-03|115.13381013416108|
|2019-04|122.79476238432385|
|2019-05|126.34636306762695|
|2019-06|131.99400062561034|
|2019-07|  138.102727023038|
|2019-08| 136.

In [43]:
AMZN.analysis.get_daily_return("month")

+-------+------------------+------------------+--------------------+--------------------+
|   Date|        Close_mean|         Open_mean|                diff|             daily_r|
+-------+------------------+------------------+--------------------+--------------------+
|2017-01| 63.19199962615967|63.185500144958496|0.006499481201174717|1.028634921977950...|
|2017-02| 64.11368440326892| 64.13473711515728|  -0.021052711888359|-3.28257553321809...|
|2017-03| 64.84130494490914| 64.76434906669284| 0.07695587821629601|0.001188244448146134|
|2017-04| 66.17157946134868| 66.23894781815379|-0.06736835680510467|-0.00101705052728...|
|2017-05| 68.91727308793502| 68.82818222045898| 0.08909086747603112|0.001294395182349...|
|2017-06|  70.5181815407493| 70.56181820956144|-0.04363666881214101|-6.18417579356367...|
|2017-07| 72.01050033569337| 71.84349975585937| 0.16700057983399574|0.002324505075636653|
|2017-08| 72.81695755668308|  72.7156518023947| 0.10130575428837574|0.001393176733994...|
|2017-09| 

In [44]:
AMZN.analysis.get_daily_return("year")

+----+------------------+------------------+--------------------+--------------------+
|Date|        Close_mean|         Open_mean|                diff|             daily_r|
+----+------------------+------------------+--------------------+--------------------+
|2017| 71.98402421502954| 71.95430287516925|  0.0297213398602878|4.130585478932400...|
|2018|101.03398411967365|101.12235092831799|-0.08836680864433788|-8.73860307173613...|
|2019|130.38202400813026|130.33904787093874| 0.04297613719151627|3.297257260469717E-4|
|2020| 190.8616180419922|190.76480678836674| 0.09681125362544662|5.074901144258145E-4|
+----+------------------+------------------+--------------------+--------------------+

